In [12]:
import networkx as nx
import matplotlib.pyplot as plt

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import Aer
from qiskit.circuit import Parameter
from qiskit.visualization import plot_histogram
from functools import reduce

import numpy as np
from scipy.optimize import minimize

In [13]:
GateX = np.array([[0, 1],[1, 0]])
GateY = np.array([[0, -1j],[1j, 0]])
GateZ = np.array([[1, 0],[0, -1]])

# 定义σ+和σ-矩阵
sigma_plus = np.array([[0, 1], [0, 0]])
sigma_minus = np.array([[0, 0], [1, 0]])

def add_in_target(num_qubits, target_qubit, gate=np.array([[1, 0],[0, -1]])):
    H = np.eye(2 ** (target_qubit))
    H = np.kron(H, gate)
    H = np.kron(H, np.eye(2 ** (num_qubits - 1 - target_qubit)))
    return H

def calculate_hamiltonian(v, w):
    n = len(v[0])
    m = len(v)
    hamiltonian = np.zeros((2**n, 2**n))

    for i in range(m):
        term1 = reduce(np.kron, [np.linalg.matrix_power(sigma_plus, v[i][j]) for j in range(n)])
        term2 = reduce(np.kron, [np.linalg.matrix_power(sigma_minus, w[i][j]) for j in range(n)])
        term3 = reduce(np.kron, [np.linalg.matrix_power(sigma_plus, w[i][j]) for j in range(n)])
        term4 = reduce(np.kron, [np.linalg.matrix_power(sigma_minus, v[i][j]) for j in range(n)])

        hamiltonian += term1 @ term2 + term3 @ term4

    return hamiltonian

def first_nonzero_index(arr, total_bits=3):
    for i, num in enumerate(arr):
        if num != 0:
            binary_repr = format(i, '0' + str(total_bits) + 'b')
            return binary_repr
        
def gnrt_Hd():
  Hd = np.zeros((2**3, 2**3)).astype(np.complex128)
  Hd += add_in_target(3, 0, GateZ)
  Hd += add_in_target(3, 1, GateZ)
  Hd += add_in_target(3, 2, GateZ)
  return Hd

In [14]:
v = np.array([[1, 0, 0],
              [0, 1, 0],
              [0, 0, 1]])
w = np.array([[0, 1, 0], 
              [0, 0, 1],
              [1, 0, 0]])

In [18]:
def generate_Hp():
    Hp = np.zeros((2**3, 2**3))
    for i in range(3):
        Hp += (add_in_target(3, i) - np.eye(2**3)) / 2
    return Hp
    


Hd = calculate_hamiltonian(v, w)
Hp = generate_Hp()

print(Hd)
print(Hp)

AB = np.dot(Hd, Hp)
BA = np.dot(Hp, Hd)



# 验证是否对易
print("对易" if np.array_equal(AB, BA) else "不对易")

[[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 0.]
 [0. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]
[[ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. -1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. -1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. -2.  0.  0.  0.  0.]
 [ 0.  0.  0.  0. -1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. -2.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -2.  0.]
 [ 0.  0.  0.  0.  0.  0.  0. -3.]]
对易


In [23]:
# x1+x2+x3=c 的 Hd
eigenvalues, eigenvectors = np.linalg.eig(Hd)
print(eigenvalues)
# 输出结果
for i in range(len(eigenvalues)):
  print("\nEigenvalue:")
  print(f'{eigenvalues[i].real:.4f}')
  print("Eigenvector:")
  print([f'{num:.2f}' for num in eigenvectors[:, i].real])

[-1.  2. -1.  2. -1. -1.  0.  0.]

Eigenvalue:
-1.0000
Eigenvector:
['0.00', '0.00', '0.00', '0.41', '0.00', '0.41', '-0.82', '0.00']

Eigenvalue:
2.0000
Eigenvector:
['0.00', '-0.00', '0.00', '0.58', '0.00', '0.58', '0.58', '0.00']

Eigenvalue:
-1.0000
Eigenvector:
['0.00', '0.77', '-0.38', '-0.14', '-0.38', '-0.14', '0.28', '0.00']

Eigenvalue:
2.0000
Eigenvector:
['0.00', '-0.55', '-0.55', '-0.18', '-0.55', '-0.18', '-0.18', '0.00']

Eigenvalue:
-1.0000
Eigenvector:
['0.00', '0.42', '0.36', '-0.18', '-0.78', '-0.04', '0.22', '0.00']

Eigenvalue:
-1.0000
Eigenvector:
['0.00', '-0.24', '0.04', '-0.64', '0.20', '0.70', '-0.07', '0.00']

Eigenvalue:
0.0000
Eigenvector:
['1.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00']

Eigenvalue:
0.0000
Eigenvector:
['0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '1.00']


In [22]:
# ∑σz x1+x2+x3 的 C
C = gnrt_Hd()
print(C.real)
print("C的本征值 和 对应本征向量")
eigenvalues, eigenvectors = np.linalg.eig(C)
print(eigenvalues.real)
# 输出结果
for i in range(len(eigenvalues)):
  print("\nEigenvalues:")
  print(f'{eigenvalues[i].real:.4f}')
  print("Eigenvectors:")
  print([f'{num:.2f}' for num in eigenvectors[:, i].real])

[[ 3.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. -1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0. -3.]]
C的本征值 和 对应本征向量
[ 3.  1.  1. -1.  1. -1. -1. -3.]

Eigenvalues:
3.0000
Eigenvectors:
['1.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00']

Eigenvalues:
1.0000
Eigenvectors:
['0.00', '1.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00']

Eigenvalues:
1.0000
Eigenvectors:
['0.00', '0.00', '1.00', '0.00', '0.00', '0.00', '0.00', '0.00']

Eigenvalues:
-1.0000
Eigenvectors:
['0.00', '0.00', '0.00', '1.00', '0.00', '0.00', '0.00', '0.00']

Eigenvalues:
1.0000
Eigenvectors:
['0.00', '0.00', '0.00', '0.00', '1.00', '0.00', '0.00', '0.00']

Eigenvalues:
-1.0000
Eigenvectors:
['0.00', '0.00', '0.00', '0.00', '0.00', '1.00', '0.00', '0.00']

Eigenvalues:
-1.0000
Eigenvectors:
['0.00', '0.00', '0.00', '0.00',